## Webscrapping NIVEL 2 - Extracción Vertical

In [1]:
import numpy as np
import pandas as pd

### 2.1.- Extracción Vertical + Crawling en TripAdvisor con Scrapy

In [2]:
import numpy as np
import pandas as pd
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider,Rule
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader

In [3]:
class Hotel(Item):
    nombre = Field()
    precio = Field()
    descripcion = Field()
    amenities = Field()
    
class TripAdvisor(CrawlSpider):
    name = "Hoteles"
    custom_settings={
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/80.0.3987.149 Safari/537.36",
            "CLOSESPIDER_PAGECOUNT":50 
        }
    start_urls = ["https://www.tripadvisor.es/Hotels-g187514-Madrid-Hotels.html"]
    download_delay = 2 # se statea para evitar baneos
    
    # Debemos definir las reglas de extracción vertical: a dónde tiene que navegar la herramienta para extraer info.
    # Para ello, debemos averiguar y definir un patrón que coincida con todas las sub urls que queremos trabajar.
    
    rules = (
        Rule(
            LinkExtractor(
                allow = r'/Hotel_Review-' # se toma de la url
            ), follow =True, callback = "parse_item" # para las url que cumplan con la variable allow se aplicrá la función de abajo
        ),
    )
    
    #MAPCOMPOSE preprocesa la información extraida -limpiar y modificar los datos parseados- antes de guardarla en un archivo.
    #En la función de abajo tomamos el texto definido abajo y le aplicamos QUITARSIMBOLO, que quita el símbolo que aparece
    #en la extracción entre el precio y el símbolo del €.
    
    def quitarsimbolo(self,texto):
        nuevoTexto = texto.replace('\xa0','')
        nuevoTexto = nuevoTexto.replace('\n','').replace('\r','')
        return nuevoTexto
    
    def parse_item(self,response):
        sel = Selector(response)
        item = ItemLoader(Hotel(),sel)
        
        item.add_xpath('nombre','//h1[@id="HEADING"]/text()')
        item.add_xpath('precio','//div[@class="CEf5oHnZ"]/text()',
                      MapCompose(self.quitarsimbolo)) 
        # añadimos self. porque la función quitarsimbolo está fuera de esta función pero está dentro de la clase TripAdvisor
        
        item.add_xpath('descripcion','//div[contains(@class,"_2f_ruteS _1bona3Pu _2-hMril5")]/div[1]/text()')
        item.add_xpath('amenities','//div[contains(@class,"_2rdvbNSg")]/text()')
        
        yield item.load_item()

In [4]:
from scrapy.spiders import Spider
from scrapy.crawler import CrawlerProcess

if __name__ == "__main__": # Código que se va a ejecutar al dar clic en RUN
        process = CrawlerProcess()
        process.crawl(TripAdvisor) # Nombre de la clase de mi Spider
        process.start()

2020-12-25 13:48:11 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-25 13:48:11 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-12-25 13:48:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-12-25 13:48:11 [scrapy.crawler] INFO: Overridden settings:
{'CLOSESPIDER_PAGECOUNT': 50}
2020-12-25 13:48:11 [scrapy.extensions.telnet] INFO: Telnet Password: b754552629134a09
2020-12-25 13:48:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.closespider.CloseSpider',
 'scrapy.extensions.logstats.LogStats']
2020-12-25 13:48:12 [scrapy.middleware] INFO: Enabled downloader middlew

2020-12-25 13:48:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d15235805-Reviews-Hotel_Riu_Plaza_Espana-Madrid.html> (referer: https://www.tripadvisor.es/Hotels-g187514-Madrid-Hotels.html)
2020-12-25 13:48:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d15235805-Reviews-Hotel_Riu_Plaza_Espana-Madrid.html>
{'amenities': ['Aparcamiento público de pago cerca',
               'Internet de alta velocidad gratuito (wifi)',
               'Piscina',
               'Gimnasio / Sala de entrenamiento',
               'Bar/salón',
               'Servicio de taxi',
               'Centro de negocios con acceso a Internet',
               'Instalaciones para conferencias',
               'Wifi',
               'Toallas para la piscina y la playa',
               'Piscina en la azotea',
               'Piscina con vistas',
               'Piscina exterior',
               'Restaurante',
          

2020-12-25 13:48:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d4719800-Reviews-Only_YOU_Boutique_Hotel_Madrid-Madrid.html> (referer: https://www.tripadvisor.es/Hotels-g187514-Madrid-Hotels.html)
2020-12-25 13:48:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d4719800-Reviews-Only_YOU_Boutique_Hotel_Madrid-Madrid.html>
{'amenities': ['Aparcamiento público de pago cerca',
               'Internet de alta velocidad gratuito (wifi)',
               'Gimnasio / Sala de entrenamiento',
               'Bar/salón',
               'Servicio de taxi',
               'Centro de negocios con acceso a Internet',
               'Salas de reuniones',
               'Envoltura corporal',
               'Wifi',
               'Cafetería',
               'Restaurante',
               'Desayuno disponible',
               'Servicio de desayuno en la habitación',
               'Café instantáneo de cort

2020-12-25 13:48:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d6373051-Reviews-Air_Rooms_Madrid_Airport_by_Premium_Traveller-Madrid.html> (referer: https://www.tripadvisor.es/Hotels-g187514-Madrid-Hotels.html)
2020-12-25 13:48:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d6373051-Reviews-Air_Rooms_Madrid_Airport_by_Premium_Traveller-Madrid.html>
{'amenities': ['Aparcamiento',
               'Internet de alta velocidad gratuito (wifi)',
               'Wifi',
               'Desayuno bufé',
               'Espacio para guardar el equipaje',
               'Hotel de no fumadores',
               'Recepción 24 horas',
               'Registro de entrada y salida exprés',
               'Aire acondicionado',
               'Mesa de escritorio',
               'Caja fuerte',
               'Teléfono',
               'Cafetera/tetera',
               'Minibar',
               'Televisor 

2020-12-25 13:48:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-Travelodge_Madrid_Torrelaguna-Madrid.html> (referer: https://www.tripadvisor.es/Hotels-g187514-Madrid-Hotels.html)
2020-12-25 13:48:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-Travelodge_Madrid_Torrelaguna-Madrid.html>
{'amenities': ['Aparcamiento privado de pago dentro del recinto',
               'Wifi',
               'Sauna',
               'Bar/salón',
               'Alquiler de bicicletas',
               'Alquiler de coches',
               'Espacio para guardar el equipaje',
               'Conserje',
               'Aparcamiento',
               'Wifi de pago',
               'Cafetería',
               'Desayuno bufé',
               'Bar de tapas',
               'Máquina expendedora',
               'Hotel de no fumadores',
               'Recepción 24 horas',
               

...

2020-12-25 13:50:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or435-Travelodge_Madrid_Torrelaguna-Madrid.html> (referer: https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or445-Travelodge_Madrid_Torrelaguna-Madrid.html)
2020-12-25 13:50:10 [scrapy.core.engine] INFO: Closing spider (closespider_pagecount)
2020-12-25 13:50:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or435-Travelodge_Madrid_Torrelaguna-Madrid.html>
{'amenities': ['Aparcamiento privado de pago dentro del recinto',
               'Wifi',
               'Sauna',
               'Bar/salón',
               'Alquiler de bicicletas',
               'Alquiler de coches',
               'Espacio para guardar el equipaje',
               'Conserje',
               'Aparcamiento',
               'Wifi de pago',
               'Cafetería',
               'Desayuno bufé',
            

2020-12-25 13:50:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d231637-Reviews-or2465-Hotel_Liabeny-Madrid.html> (referer: https://www.tripadvisor.es/Hotel_Review-g187514-d231637-Reviews-or2485-Hotel_Liabeny-Madrid.html)
2020-12-25 13:50:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d231637-Reviews-or2465-Hotel_Liabeny-Madrid.html>
{'amenities': ['Servicio de aparcacoches',
               'Internet de alta velocidad gratuito (wifi)',
               'Gimnasio / Sala de entrenamiento',
               'Bar/salón',
               'Pub/DJ',
               'Centro de negocios con acceso a Internet',
               'Instalaciones para conferencias',
               'Vigilancia 24 horas',
               'Aparcamiento privado de pago dentro del recinto',
               'Wifi',
               'Sauna',
               'Cafetería',
               'Restaurante',
               'Desayuno bufé',
    

2020-12-25 13:50:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d12560161-Reviews-or310-Axel_Hotel_Madrid-Madrid.html> (referer: https://www.tripadvisor.es/Hotel_Review-g187514-d12560161-Reviews-or330-Axel_Hotel_Madrid-Madrid.html)
2020-12-25 13:50:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d12560161-Reviews-or310-Axel_Hotel_Madrid-Madrid.html>
{'amenities': ['Aparcamiento privado de pago cerca',
               'Internet de alta velocidad gratuito (wifi)',
               'Piscina',
               'Gimnasio / Sala de entrenamiento',
               'Bar/salón',
               'Servicio de taxi',
               'Salas de reuniones',
               'Balneario',
               'Wifi',
               'Toallas para la piscina y la playa',
               'Piscina en la azotea',
               'Piscina con vistas',
               'Piscina para adultos',
               'Piscina exterior',
  

2020-12-25 13:50:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or430-Travelodge_Madrid_Torrelaguna-Madrid.html> (referer: https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or445-Travelodge_Madrid_Torrelaguna-Madrid.html)
2020-12-25 13:50:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d497864-Reviews-or430-Travelodge_Madrid_Torrelaguna-Madrid.html>
{'amenities': ['Aparcamiento privado de pago dentro del recinto',
               'Wifi',
               'Sauna',
               'Bar/salón',
               'Alquiler de bicicletas',
               'Alquiler de coches',
               'Espacio para guardar el equipaje',
               'Conserje',
               'Aparcamiento',
               'Wifi de pago',
               'Cafetería',
               'Desayuno bufé',
               'Bar de tapas',
               'Máquina expendedora',
               'Hotel de no f

2020-12-25 13:50:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.es/Hotel_Review-g187514-d1177124-Reviews-or1220-ILUNION_Atrium-Madrid.html> (referer: https://www.tripadvisor.es/Hotel_Review-g187514-d1177124-Reviews-or1235-ILUNION_Atrium-Madrid.html)
2020-12-25 13:50:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.tripadvisor.es/Hotel_Review-g187514-d1177124-Reviews-or1220-ILUNION_Atrium-Madrid.html>
{'amenities': ['Servicio de aparcacoches',
               'Internet de alta velocidad gratuito (wifi)',
               'Piscina exterior',
               'Gimnasio / Sala de entrenamiento',
               'Bar/salón',
               'Alquiler de coches',
               'Centro de negocios con acceso a Internet',
               'Instalaciones para conferencias',
               'Aparcamiento privado de pago dentro del recinto',
               'Wifi',
               'Cafetería',
               'Restaurante',
               'Desayuno bufé',
             

<img src='c1.PNG'>